In [42]:
import main_Stuart as ms

import main_functions_Stewart_platform as mf

import numpy as np

import pandas as pd

import torch

from torch import nn

import torch.nn.functional as F

from tqdm import tqdm

In [2]:
# Класс для оставляния датасета внутри класса
# решить вопрос с инициализацией, тк нельзя в инициализации возвращать что либо.
class Model_Stuart_from_model():

    def __init__(self, model, n):
        # Создание модели стюарта
        self.model = model
        # Сохранение результатов
        self.results = np.array([])
        # Сохранение промежуточных результатов
        self.intermediate_result = np.array([])
        # Создание цикла для формирования датасета
        size = n
        while n != 0:
            # Генерация перемещений
            coordinates = (np.random.rand(
                3)*self.model.max_len)-self.model.max_len/2
            # Генерация углов поворота
            angle = 90*np.random.rand(3)
            # Сохранение координат верхней платформы
            self.model.change_of_position_upper_platform(alfa=angle[0], betta=angle[1], gamma=angle[2],

                                                         x=coordinates[0], y=coordinates[1], z=coordinates[2])
            # Расчитываем углы
            self.model.angle()
            # Расчитываем длинны
            self.model.len_leg()

            if sum(self.model.test_lens) == 6 and sum(sum(self.model.test_angle)) == 12:

                self.intermediate_result = np.concatenate(
                    (np.concatenate((angle, coordinates)), self.model.len))

                self.intermediate_result = self.intermediate_result.reshape(
                    1, -1)

                self.results = np.append(
                    self.results, self.intermediate_result)

                n -= 1

            else:
                print('Длинны ног', self.model.len, self.model.test_lens, 'Углы наклона ног',
                      self.model.angle_lens, self.model.test_angle, '--------------------', sep='\n')
                self.model.coordinates_upper_platform = mf.calculation_new_coordinates(
                    self.model._Stuart__inv_matrix, self.model.coordinates_upper_platform).reshape(-1, 3)

        self.results = self.results.reshape(size, -1)


In [34]:
# Класс для выведения датасета как переменной
class Model_Stuart():

    def __init__(self, model, n):
        # Создание модели стюарта
        self.model = model
        # Сохранение результатов
        self.results = np.array([])
        # Сохранение промежуточных результатов
        self.intermediate_result = np.array([])
        # Создание цикла для формирования датасета
        self.size = n

        self.n = n

        self.x = np.array([])

        self.y = np.array([])

    def dataset(self,pandas_df =True , prints=False):
        while self.n != 0:
            # Генерация перемещений
            coordinates = 0.1*((np.random.rand(3)*self.model.max_len)-self.model.max_len/2)
            # Генерация углов поворота
            angle = 90*np.random.rand(3) - 45
            # Сохранение координат верхней платформы
            self.model.change_of_position_upper_platform(alfa=angle[0], betta=angle[1], gamma=angle[2],

                                                         x=coordinates[0], y=coordinates[1], z=coordinates[2])
            # Расчитываем углы
            self.model.angle()
            # Расчитываем длинны
            self.model.len_leg()

            if sum(self.model.test_lens) == 6 and sum(sum(self.model.test_angle)) == 12:

                self.intermediate_result = np.append((np.concatenate((np.concatenate(
                    (angle, coordinates)), self.model.len))), self.model.angle_lens.reshape(1, -1))

                self.intermediate_result = self.intermediate_result.reshape(
                    1, -1)

                self.results = np.append(
                    self.results, self.intermediate_result)

                self.n -= 1

            else:
                if prints == True:
                    print('Длинны ног', self.model.len, self.model.test_lens, 'Углы наклона ног',
                          self.model.angle_lens, self.model.test_angle, '--------------------', sep='\n')
                self.model.coordinates_upper_platform = mf.calculation_new_coordinates(
                    self.model._Stuart__inv_matrix, self.model.coordinates_upper_platform).reshape(-1, 3)

        #self.results = self.results.reshape(size,-1)

        self.results = self.results.reshape(self.size, -1)

        self.x = self.results[:,:6]

        self.y = self.results[:,6:]

        columns_my = ['Угол поворота Х', 'Угол поворта Y', 'Угол поворота Z',
                      'Перемещение по Х', 'Перемещение по Y', 'Перемещение по Z',
                      'Длинна 1 ноги', 'Длинна 2 ноги', 'Длинна 3 ноги', 'Длинна 4 ноги', 'Длинна 5 ноги', 'Длинна 6 ноги',
                      'Угол поворота 1 ноги у нижней платформы', 'Угол поворота 1 ноги у верхней платформы', 'Угол поворота 2 ноги у нижней платформы', 'Угол поворота 2 ноги у верхней платформы', 'Угол поворота 3 ноги у нижней платформы', 'Угол поворота 3 ноги у верхней платформы',
                      'Угол поворота 4 ноги у нижней платформы', 'Угол поворота 4 ноги у верхней платформы', 'Угол поворота 5 ноги у нижней платформы', 'Угол поворота 5 ноги у верхней платформы', 'Угол поворота 6 ноги у нижней платформы', 'Угол поворота 6 ноги у верхней платформы']
        
        if pandas_df:
            return pd.DataFrame(self.results, columns=columns_my),self.x,self.y
        else:
            return self.x,self.y


In [35]:
import numpy as np
import main_Stuart as ms
import sys

sys.path.append(sys.path[0].replace('StewartTest', 'StewartLibrary/'))


model = ms.Stuart(coordinates_global=[0, 0, 0], min_len=3000,
                  max_len=7000, angles=45)
model.coordinate_lower_platform(alfa=0, betta=0, gamma=0,
                                x=0, y=0, z=0, R_lower=4000)
model.coordinate_upper_platform(alfa=0, betta=0, gamma=0,
                                x=0, y=0, z=0, R_upper=3000)


Вы задали
Координаты глобальной системы:[0, 0, 0]
Минимальная и максимальная длинна ноги:(3000, 7000)
Минимальный и максимальный наклон ног:(45, 135)


In [36]:
e = Model_Stuart(model,100)
df,x,y = e.dataset(pandas_df=True)

In [37]:
df

,Угол поворота Х,Угол поворта Y,Угол поворота Z,Перемещение по Х,Перемещение по Y,Перемещение по Z,Длинна 1 ноги,Длинна 2 ноги,Длинна 3 ноги,Длинна 4 ноги,...,Угол поворота 2 ноги у нижней платформы,Угол поворота 2 ноги у верхней платформы,Угол поворота 3 ноги у нижней платформы,Угол поворота 3 ноги у верхней платформы,Угол поворота 4 ноги у нижней платформы,Угол поворота 4 ноги у верхней платформы,Угол поворота 5 ноги у нижней платформы,Угол поворота 5 ноги у верхней платформы,Угол поворота 6 ноги у нижней платформы,Угол поворота 6 ноги у верхней платформы
0,8.905242,43.004780,3.093423,-166.874188,44.490117,-45.813773,3368.833125,5250.037712,6525.394863,6433.101967,...,93.898230,49.450698,80.780346,49.851579,81.824169,49.157031,102.408202,46.352517,81.517963,46.020604
1,-43.325308,-5.868914,4.406292,-83.879622,-294.271504,-200.231393,3542.992773,5469.900405,6677.121963,6471.560270,...,76.137114,45.812799,105.987382,45.415198,83.680753,49.218436,87.820751,51.215631,112.904480,45.771788
2,22.472426,7.183984,5.499002,-110.752181,251.273438,-209.901977,4238.266954,6025.680010,6831.049663,6218.661159,...,73.431410,51.784265,90.121273,50.184039,84.404819,45.193219,95.108894,45.248910,80.192325,49.209375
3,-3.631298,-18.032527,-12.054502,-249.966694,-217.474762,-70.808596,3201.896095,5443.688035,6951.156175,6900.855316,...,82.250938,51.104685,96.675755,51.929679,98.938119,50.915184,109.751840,46.563653,97.879305,49.196757
4,7.450780,15.964322,17.314394,224.689948,43.302765,-93.544241,4242.543467,6055.404819,6833.058390,6168.038095,...,73.398423,47.801267,82.278683,46.728060,78.228119,46.210707,83.136405,50.575077,72.556469,50.971201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-35.697038,5.321320,12.848073,-105.964956,-41.130909,11.902739,4180.864489,5753.413368,6868.710400,6755.328434,...,48.999367,58.126041,61.110029,54.186554,81.006971,54.438896,77.560434,68.449926,60.544493,87.063806
96,24.006497,-29.147287,-37.283245,267.422160,-226.967478,159.536608,3176.985450,3117.952968,4905.928441,6227.860216,...,59.580631,76.155811,72.245865,55.354066,79.967859,46.609710,70.251804,61.062323,56.996178,80.882789
97,10.992499,9.117862,-1.260950,-10.721281,69.085011,-207.464420,4001.234863,3781.664185,5243.938083,6511.964807,...,57.973717,80.440574,75.347919,60.955202,81.741959,50.387547,63.892687,60.088104,51.164236,70.702630
98,3.981931,-5.327420,31.535514,133.004282,-52.336206,254.618890,3668.104058,4828.857786,6077.600620,6379.162244,...,61.973733,89.374454,95.259887,58.067753,91.801210,45.217959,64.227034,55.577935,51.819973,68.239015


In [45]:
#Задаем 3 слоя 1)линейный с логарифмической функцией активации 
# 2) билинейный слой с логарифмической функцией активации 3)линейный слой с логарифмической функцией активации
# 
network = nn.Sequential(
    nn.Linear(6,36),
    nn.Softmax(),
    nn.Linear(36, 144),
    nn.Softmax(),
    nn.Linear(144, 18),
    nn.Softmax(),
)
optimizer = torch.optim.Adam(network.parameters())

In [46]:
x = torch.Tensor(x)
y = torch.Tensor(y)

In [49]:
losses = []
for i in range(100):
    for j in tqdm(range(4000)):
        y_pred = network(x)
        loss = F.mse_loss(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())


100%|██████████| 4000/4000 [00:03<00:00, 1152.41it/s]


In [50]:
losses

[9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 9828919.0,
 982

In [51]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot()